# Introduction

In this notebook we do a basic EDA on the image data and features revealed with the help of the Detectron2 Library

In [ ]:
# import some common libraries
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import torch

%matplotlib inline


# import some common detectron2 utilities
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_test_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.config import get_cfg
from detectron2.utils.visualizer import ColorMode
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
%matplotlib inline

First we must register our images and annotation files into the Detectron2 network

In [ ]:
register_coco_instances("train_lane_cone_detector", {}, "../test/testdata/train/train.json", "../test/testdata/train")

We can then load in the saved metadata and images

In [ ]:
train_dataset_metadata = MetadataCatalog.get("train_lane_cone_detector")
train_dataset_dicts = DatasetCatalog.get("train_lane_cone_detector")

Let's take a look at what the metadata contains

In [ ]:
train_dataset_metadata

As we can see it contains a lot of information about our dataset. Namely the location of the annotation files and the location of the images. It also was able to autodetect that are images are annotated in the COCO format (Our .json file is based of the COCO JSON captioning). Lastly it knows the possible labels our objects have (lane and cone) and has assigned integers to them for the Detectron2 network to use in calculating metrics.

Let's next take a look at the dictionary saved from the DatasetCatalog

In [ ]:
train_dataset_dicts

First we notice that it is saved into a list where each element has the same format. Let's take a look at an element

In [ ]:
train_dataset_dicts[0]

Each element seems to be talking about a specific image file in dictionary format. The dictionary contains information about the image name, it's height and width and also has an image ID attached to it. It's also contains a lot of details about the annotations, specifically the boundary boxes, segmentations and the category ID each of them belong to

Let's plot one of the images now

In [ ]:
img = cv2.imread(train_dataset_dicts[0]["file_name"])
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float)
fig = plt.figure(figsize = (20,5))
ax = fig.add_subplot(111)
ax.imshow(img/255, interpolation='none')
ax.set_title('Training Image')
ax.axis("off")
fig.show()
fig.tight_layout()

We can see that the image does indeed have a cone and lane object. 

In [ ]:
img.shape

We can also see the image indeed is a 360x1280 image. The 3 reflects 3 channels which is true since this is a BGR image

Let's draw the boundary box and segmentation results on top of this

In [ ]:
img = cv2.imread(train_dataset_dicts[0]["file_name"])
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float)
visualizer = Visualizer(img[:, :, ::-1], metadata=train_dataset_metadata, scale=1)
vis = visualizer.draw_dataset_dict(train_dataset_dicts[0])
fig = plt.figure(figsize = (20,5))
ax = fig.add_subplot(111)
ax.imshow(vis.get_image()[:, :, ::-1]/255, interpolation='none')
ax.set_title('Training Image with Labels')
ax.axis("off")
fig.show()
fig.tight_layout()

We can see the iamge has appropriate boundary boxes and segmentations surrounding it as well

The Detectron2 Network will be trained on such images and then finally use an inference on an unknown set of images to make a video